### Parsing CDS sequences from .gbk files
Here we take the output from antismash and create a dataframe with sequence information for future alignment.

In [ ]:
# Parsing CDS sequences from .gbk files in /home/pedro/antismash/resultados/all_BGCs
from Bio import SeqIO
import pandas as pd
import glob
import os

gbk_files = glob.glob("/home/pedro/antismash/resultados/all_BGCs/*.gbk")

results = []

for gbk in gbk_files:
    for record in SeqIO.parse(gbk, "genbank"):

        for feature in record.features:

            if feature.type not in ["CDS", "aSDomain"]:
                continue

            qualifiers = feature.qualifiers

            feature_id = (
                qualifiers.get("locus_tag", [None])[0]
                or qualifiers.get("protein_id", [None])[0]
                or qualifiers.get("gene", [None])[0]
                or qualifiers.get("ID", [None])[0]
                or "unknown"
            )

            if feature.type == "CDS":
                if "translation" in qualifiers:
                    sequence = qualifiers["translation"][0]
                else:
                    sequence = feature.extract(record.seq).translate(to_stop=True)

            elif feature.type == "aSDomain":
                sequence = str(feature.extract(record.seq))

            results.append({
                "gbk_file": os.path.basename(gbk),
                "record_id": record.id,
                "feature_type": feature.type,
                "feature_id": feature_id,
                "start": int(feature.location.start),
                "end": int(feature.location.end),
                "strand": feature.location.strand,
                "sequence": str(sequence)
            })


In [60]:
df = pd.DataFrame(results)
df

,gbk_file,record_id,feature_type,feature_id,start,end,strand,sequence
0,MGYG000296065_60.region001.gbk,MGYG000296065_60,aSDomain,ctg60_2,166,613,1,ATGAACGTTATACTATTAGAAAAGACTCGTAATCTAGGTGACCTAG...
1,MGYG000296065_60.region001.gbk,MGYG000296065_60,CDS,ctg60_2,166,616,1,MNVILLEKTRNLGDLGEQVSVKAGYGRNFLIPLGKAVPATETNVKH...
2,MGYG000296065_60.region001.gbk,MGYG000296065_60,CDS,ctg60_3,567,738,-1,MTNYQSITLPPEKPIARSTKCDELLIVADDTAYRELESSNAYSGTT...
3,MGYG000296065_60.region001.gbk,MGYG000296065_60,CDS,ctg60_4,779,2177,1,MSDFERERPAPLDFDTSALKVPPHSLEAEQSVLGGLMLSNAAFDDV...
4,MGYG000296065_60.region001.gbk,MGYG000296065_60,aSDomain,ctg60_4,842,2144,1,CCCCCGCATTCATTGGAAGCTGAGCAATCTGTGCTGGGTGGTTTGA...
...,...,...,...,...,...,...,...,...
2573,MGYG000296041_38.region001.gbk,MGYG000296041_38,CDS,ctg38_24,6703,8629,-1,MPEDAENNPVPAILEYIPYRKRDNTRSRDALNHPYFAGHGYASIRV...
2574,MGYG000296041_38.region001.gbk,MGYG000296041_38,aSDomain,ctg38_24,7120,8617,-1,GCCGAGAACAATCCGGTGCCGGCGATCCTCGAATACATCCCCTATC...
2575,MGYG000296041_38.region001.gbk,MGYG000296041_38,aSDomain,ctg38_24,8275,8530,-1,CCCTATTTTGCCGGGCACGGCTATGCCAGCATCCGGGTGGATATAC...
2576,MGYG000296041_38.region001.gbk,MGYG000296041_38,CDS,ctg38_25,8839,10153,-1,MTDHCGYIDATRVRRRLVDLIRIPSVTGQEDAVIARLAEWLNELGV...


In [61]:
# Check for repetitive feature IDs
df["feature_id"].value_counts()

feature_id
ctg8_28     29
ctg48_1     19
ctg72_16    18
ctg23_6     16
ctg23_4     14
            ..
ctg60_22     1
ctg60_21     1
ctg60_20     1
ctg60_19     1
ctg60_18     1
Name: count, Length: 1431, dtype: int64

In [62]:
# Making a new column that combines record_id and feature_id to ensure uniqueness
df["start_end"] = df["start"].astype(str) + "-" + df["end"].astype(str)
df["unique_id"] = df["feature_id"] + "_" + df["start_end"]
df["unique_id"].value_counts()

unique_id
ctg23_4_14260-14725    2
ctg8_28_34959-35409    2
ctg48_1_4233-4701      2
ctg8_28_39546-40002    2
ctg48_1_8850-9318      2
                      ..
ctg112_6_5751-6657     1
ctg112_6_5805-6594     1
ctg112_7_6730-6868     1
ctg1_121_182-530       1
ctg112_1_1-649         1
Name: count, Length: 2573, dtype: int64

In [63]:
# View dataframe only with ctg48_1_4233-4701
df[df["unique_id"] == "ctg48_1_4233-4701"]

,gbk_file,record_id,feature_type,feature_id,start,end,strand,sequence,start_end,unique_id
771,MGYG000296065_48.region001.gbk,MGYG000296065_48,aSDomain,ctg48_1,4233,4701,1,TTATCATCAAATATTAAGAAGGTGAAGGAGCAATTACGAGGTATAC...,4233-4701,ctg48_1_4233-4701
772,MGYG000296065_48.region001.gbk,MGYG000296065_48,aSDomain,ctg48_1,4233,4701,1,TTATCATCAAATATTAAGAAGGTGAAGGAGCAATTACGAGGTATAC...,4233-4701,ctg48_1_4233-4701


In [64]:
# Drop the unique_id duplicates
df = df.drop_duplicates(subset="unique_id")
df = df.drop(columns=["gbk_file", "record_id", "feature_id", "strand", "start", "end"])
df["unique_id"].value_counts()

unique_id
ctg60_2_166-613        1
ctg60_2_166-616        1
ctg60_3_567-738        1
ctg60_4_779-2177       1
ctg60_4_842-2144       1
                      ..
ctg38_24_6703-8629     1
ctg38_24_7120-8617     1
ctg38_24_8275-8530     1
ctg38_25_8839-10153    1
ctg38_25_8929-10105    1
Name: count, Length: 2573, dtype: int64

In [65]:
df = df.reindex(columns=[
    "unique_id",
    "feature_type",
    "start_end",
    "sequence"
])
df

,unique_id,feature_type,start_end,sequence
0,ctg60_2_166-613,aSDomain,166-613,ATGAACGTTATACTATTAGAAAAGACTCGTAATCTAGGTGACCTAG...
1,ctg60_2_166-616,CDS,166-616,MNVILLEKTRNLGDLGEQVSVKAGYGRNFLIPLGKAVPATETNVKH...
2,ctg60_3_567-738,CDS,567-738,MTNYQSITLPPEKPIARSTKCDELLIVADDTAYRELESSNAYSGTT...
3,ctg60_4_779-2177,CDS,779-2177,MSDFERERPAPLDFDTSALKVPPHSLEAEQSVLGGLMLSNAAFDDV...
4,ctg60_4_842-2144,aSDomain,842-2144,CCCCCGCATTCATTGGAAGCTGAGCAATCTGTGCTGGGTGGTTTGA...
...,...,...,...,...
2573,ctg38_24_6703-8629,CDS,6703-8629,MPEDAENNPVPAILEYIPYRKRDNTRSRDALNHPYFAGHGYASIRV...
2574,ctg38_24_7120-8617,aSDomain,7120-8617,GCCGAGAACAATCCGGTGCCGGCGATCCTCGAATACATCCCCTATC...
2575,ctg38_24_8275-8530,aSDomain,8275-8530,CCCTATTTTGCCGGGCACGGCTATGCCAGCATCCGGGTGGATATAC...
2576,ctg38_25_8839-10153,CDS,8839-10153,MTDHCGYIDATRVRRRLVDLIRIPSVTGQEDAVIARLAEWLNELGV...


In [66]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

records = [
    SeqRecord(
        Seq(seq),
        id=cds_id,
        description=""
    )
    for cds_id, seq in zip(df["unique_id"], df["sequence"])
]

SeqIO.write(records, "antismash_cds.faa", "fasta")


2573